In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1" 
import ast
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras import backend as K
from keras import Model
from keras import optimizers
from keras.legacy import interfaces
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D, BatchNormalization
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.optimizers import Adam

from keras.optimizers import SGD
from keras import callbacks
from data import *
from keras_helper import *

from se_densenet import *
# from keras.applications.densenet import preprocess_input

Using TensorFlow backend.


In [2]:
def preprocess_input(x):
    x /= 127.5
    x -= 1.
    return x

In [3]:
TOT = 49673580
EPOCHS = 50
size = 64
batchsize = 32
lw = 6
channel = 1
STEPS = TOT / EPOCHS / batchsize / 3
NCATS = 340

In [4]:
K.clear_session()

# model = DenseNet201(input_shape=(size, size, 1), weights=None, classes=NCATS)
model = SEDenseNet(input_shape=(size, size, 1), classes=NCATS, depth = 40)

model.compile(optimizer=Adam(lr=0.002), loss='categorical_crossentropy',
              metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 24)   216         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 24)   96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 64, 64, 24)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [5]:
valid_df = pd.read_csv('../input/valid.csv')
x_valid = df_to_image_array_xd(valid_df, size, lw=lw, 
                               preprocess_input=preprocess_input, channel=channel)
y_valid = keras.utils.to_categorical(valid_df.word, num_classes=NCATS)

In [6]:
print(x_valid.shape, y_valid.shape)
print('Validation array memory {:.2f} GB'.format(x_valid.nbytes / 1024.**3 ))

(34000, 64, 64, 1) (34000, 340)
Validation array memory 0.52 GB


In [7]:
train_datagen = image_generator_xd(size=size, batchsize=batchsize, lw=lw, 
                                   preprocess_input=preprocess_input,
                                  channel=channel)

In [8]:
model_prefix = 'sedensenet_raw_head_{}_lw{}'.format(size, lw)
fold = 9
callbks = [
    ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=5,
                      min_delta=0.005, mode='max', cooldown=3, verbose=1),
    callbacks.ModelCheckpoint("./models/{}.model".format(model_prefix),
                                monitor='val_categorical_accuracy', 
                                mode = 'max', save_best_only=True, verbose=1),
    callbacks.TensorBoard(log_dir='./log/{}'.format(model_prefix)),
]

In [ ]:
hists = []
hist = model.fit_generator(  
    train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks = callbks
)

Epoch 1/50
10349/10348 [==============================] - 2410s 233ms/step - loss: 3.2662 - categorical_crossentropy: 3.2123 - categorical_accuracy: 0.2954 - top_3_accuracy: 0.4700 - val_loss: 2.3367 - val_categorical_crossentropy: 2.2788 - val_categorical_accuracy: 0.4642 - val_top_3_accuracy: 0.6747

Epoch 00001: val_categorical_accuracy improved from -inf to 0.46418, saving model to ./models/sedensenet_raw_head_64_lw6.model
Epoch 2/50
10349/10348 [==============================] - 2410s 233ms/step - loss: 1.9865 - categorical_crossentropy: 1.9284 - categorical_accuracy: 0.5379 - top_3_accuracy: 0.7425 - val_loss: 1.9025 - val_categorical_crossentropy: 1.8458 - val_categorical_accuracy: 0.5592 - val_top_3_accuracy: 0.7621

Epoch 00002: val_categorical_accuracy improved from 0.46418 to 0.55924, saving model to ./models/sedensenet_raw_head_64_lw6.model
Epoch 3/50
10349/10348 [==============================] - 2411s 233ms/step - loss: 1.7193 - categorical_crossentropy: 1.6632 - categori

10349/10348 [==============================] - 2413s 233ms/step - loss: 1.2082 - categorical_crossentropy: 1.1634 - categorical_accuracy: 0.7119 - top_3_accuracy: 0.8710 - val_loss: 1.3462 - val_categorical_crossentropy: 1.3016 - val_categorical_accuracy: 0.6927 - val_top_3_accuracy: 0.8568

Epoch 00021: val_categorical_accuracy did not improve from 0.71147
Epoch 22/50
10349/10348 [==============================] - 2414s 233ms/step - loss: 1.2035 - categorical_crossentropy: 1.1591 - categorical_accuracy: 0.7137 - top_3_accuracy: 0.8725 - val_loss: 1.2016 - val_categorical_crossentropy: 1.1572 - val_categorical_accuracy: 0.7158 - val_top_3_accuracy: 0.8728

Epoch 00022: val_categorical_accuracy improved from 0.71147 to 0.71579, saving model to ./models/sedensenet_raw_head_64_lw6.model
Epoch 23/50
10349/10348 [==============================] - 2414s 233ms/step - loss: 1.2072 - categorical_crossentropy: 1.1629 - categorical_accuracy: 0.7124 - top_3_accuracy: 0.8711 - val_loss: 1.2174 - va

In [ ]:
model.load_weights('./models/{}.model'.format(model_prefix))

In [ ]:
test = pd.read_csv('../input/test_simplified.csv')
test.head()
x_test = df_to_image_array_xd(test, size, lw=lw, 
                              preprocess_input=preprocess_input,
                             channel=channel)
print(test.shape, x_test.shape)
print('Test array memory {:.2f} GB'.format(x_test.nbytes / 1024.**3 ))

In [ ]:
test_predictions = model.predict(x_test, batch_size=128, verbose=1)

top3 = preds2catids(test_predictions)
top3.head()
top3.shape

In [ ]:
np_classes = np.load('../input/classes.npy')
id2cat = {k: cat.replace(' ', '_') for k, cat in enumerate(np_classes)}
top3cats = top3.replace(id2cat)
top3cats.head()
top3cats.shape

In [ ]:
test['word'] = top3cats['a'] + ' ' + top3cats['b'] + ' ' + top3cats['c']
submission = test[['key_id', 'word']]
submission.head()

In [ ]:
import kaggle_util
kaggle_util.save_result(submission, 
                        '../result/{}.csv'.format(model_prefix), 
                        'quickdraw-doodle-recognition', 
                        send=True, index=False)